In [6]:
# this code is generated by the Domino Code Assist toolbar button
import domino_code_assist as dca
dca.init()

Cannot show ipywidgets in text

# Power Generation in the UK

This app visualizes energy production in the UK by source for the past day and week.

The data is provided by the Balancing Mechanism Reporting Service (BMRS), which provides operational data relating to the GB Electricity Balancing and Settlement arrangements. It is used extensively by market participants to help make trading decisions and understand market dynamics. It acts as a prompt reporting platform as well as a means of accessing historic data.

For more info: https://www.bmreports.com/bmrs/?q=help/about-us

Data is read in each day from BMRS using a scheduled Domino job, and cleaned up and visualized using an app built with Domino's Code Assistant.

In [7]:
import pandas as pd
import os

df = pd.read_csv('/mnt/data/{}/app_data.csv'.format(os.environ.get('DOMINO_PROJECT_NAME')), parse_dates=['datetime'])
df.head()

,HDF,datetime,CCGT,OIL,COAL,NUCLEAR,WIND,PS,NPSHYD,OCGT,OTHER,INTFR,INTIRL,INTNED,INTEW,BIOMASS,INTEM
0,FUELHH,2023-04-10 00:00:00,5777,0,0,4485,11002,0,138,1,161,0,0,124,0,1901,0
1,FUELHH,2023-04-10 00:30:00,4897,0,0,4488,11694,0,136,1,133,0,0,126,0,1938,0
2,FUELHH,2023-04-10 01:00:00,4186,0,0,4485,11938,0,101,1,136,0,0,102,0,1841,0
3,FUELHH,2023-04-10 01:30:00,4331,0,0,4482,12068,0,100,1,138,0,0,102,0,1746,0
4,FUELHH,2023-04-10 02:00:00,4401,0,0,4478,12227,0,99,0,146,0,0,0,0,1836,0


In [8]:
import datetime
df_today = df.loc[df["datetime"] > (datetime.datetime.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')]

df_week = df.loc[df["datetime"] > (datetime.datetime.today() - datetime.timedelta(days=7)).strftime('%Y-%m-%d %H:%M:%S')]
df_today.head()

,HDF,datetime,CCGT,OIL,COAL,NUCLEAR,WIND,PS,NPSHYD,OCGT,OTHER,INTFR,INTIRL,INTNED,INTEW,BIOMASS,INTEM
47,FUELHH,2023-04-10 23:30:00,2953,0,0,4713,12696,0,185,2,146,0,0,100,0,1337,0
48,FUELHH,2023-04-11 00:00:00,4121,0,0,4713,12692,0,185,2,295,0,0,0,0,1397,0
49,FUELHH,2023-04-11 00:30:00,4024,0,0,4714,12605,0,182,2,360,0,0,0,0,1421,0
50,FUELHH,2023-04-11 01:00:00,4383,0,0,4718,12584,0,167,21,341,0,0,0,0,1421,0
51,FUELHH,2023-04-11 01:30:00,4463,0,0,4712,13058,0,165,79,490,0,0,0,0,1423,0


In [9]:
def vertical_stack(df, columns, index_col, stack_id, value_id):
    df_out = pd.DataFrame(columns=[index_col, stack_id, value_id])
    temp = pd.DataFrame(columns=[index_col, stack_id, value_id])
    
    for c in columns:
        temp[index_col] = df[index_col]
        temp[stack_id] = c
        temp[value_id] = df[c]
        df_out = pd.concat([df_out, temp])
    
    return df_out


columns = ['CCGT', 'OIL', 'COAL', 'NUCLEAR', 'WIND', 'PS', 'NPSHYD', 'OCGT', 'OTHER', 'INTFR', 'INTIRL', 'INTNED', 'INTEW', 'BIOMASS', 'INTEM']
index_col = 'datetime'
stack_id = 'Source'
value_id = 'Production (MW)'

# 
df_today = vertical_stack(df_today, columns, index_col, stack_id, value_id)
df_today["datetime"] = df_today["datetime"].astype(str)

df_week = vertical_stack(df_week, columns, index_col, stack_id, value_id)
df_week["datetime"] = df_week["datetime"].astype(str)


### Production this Week by Source

In [10]:
from solara.express import CrossFilteredFigurePlotly
import plotly.express as px

_base_var_1 = px.area(df_week, x="datetime", y="Production (MW)", template="plotly_dark", color="Source")
_base_var_1.update_layout(margin=dict(l=0, r=0, t=40 if _base_var_1.layout.title.text else 20, b=0))
var_1 = CrossFilteredFigurePlotly(_base_var_1)

var_1

Cannot show ipywidgets in text

### Production Today: All Sources

In [11]:
from solara.express import CrossFilteredFigurePlotly
import plotly.express as px

_base_var_3 = px.area(df_today, x="datetime", y="Production (MW)", color="Source", template="plotly_dark")
_base_var_3.update_layout(margin=dict(l=0, r=0, t=40 if _base_var_3.layout.title.text else 20, b=0))
var_3 = CrossFilteredFigurePlotly(_base_var_3)

var_3

Cannot show ipywidgets in text

In [12]:
# this code is generated by the Domino Code Assist toolbar button
import domino_code_assist as dca
dca.init()

Cannot show ipywidgets in text

In [13]:
from solara import CrossFilterSelect

widget_1 = CrossFilterSelect(df_week, column="Source", configurable=False, multiple=True)
widget_1


Cannot show ipywidgets in text

### Daily Chart Data

### Weekly Chart Data

In [14]:
from domino_code_assist.deploy import Deployer
import solara


@solara.component
def Page():
    return dca.CardGridLayout([
        {'item': var_1, 'w': 11, 'h': 10, 'x': 0, 'y': 24},
        {'item': var_3, 'w': 7, 'h': 12, 'x': 0, 'y': 8},
        {'item': dca.MarkdownFromCell('7fc55319-0eef-49da-b5ee-cba30759d667'), 'w': 11, 'h': 2, 'x': 0, 'y': 20},
        {'item': dca.MarkdownFromCell('4ee0e113-a363-4d48-bf5c-1942838966bd'), 'w': 11, 'h': 2, 'x': 0, 'y': 6},
        {'item': df_today, 'w': 4, 'h': 10, 'x': 7, 'y': 10},
        {'item': df_week, 'w': 11, 'h': 7, 'x': 0, 'y': 36},
        {'item': widget_1, 'w': 11, 'h': 2, 'x': 0, 'y': 22},
        {'item': dca.MarkdownFromCell('4bc7b763-e7b4-4ae2-90ec-4861de0e57e2'), 'w': 11, 'h': 6, 'x': 0, 'y': 0},
        {'item': dca.MarkdownFromCell('1d0a0b72-ff13-49ad-bbdd-6e434abde0c3'), 'w': 4, 'h': 2, 'x': 7, 'y': 8},
        {'item': dca.MarkdownFromCell('0267a5f2-c90e-4510-b671-b95feb22a899'), 'w': 11, 'h': 2, 'x': 0, 'y': 34}
    ])

Deployer(Page)

Cannot show ipywidgets in text